In [1]:
%load_ext autoreload
%autoreload 2

from datetime import datetime
from pathlib import Path

import pandas as pd
from loguru import logger

from multiplex_pipeline.core_cutting.channel_scanner import discover_channels
from multiplex_pipeline.core_cutting.controller import CorePreparationController
from multiplex_pipeline.core_cutting.file_io import LocalFileStrategy
from multiplex_pipeline.utils.config_loaders import load_analysis_settings

c:\Users\KMK280\AppData\Local\miniconda3\envs\sdata-env\Lib\site-packages\dask\dataframe\__init__.py:31: FutureWarning: The legacy Dask DataFrame implementation is deprecated and will be removed in a future version. Set the configuration option `dataframe.query-planning` to `True` or None to enable the new Dask Dataframe implementation and silence this warning.
  warnings.warn(
c:\Users\KMK280\AppData\Local\miniconda3\envs\sdata-env\Lib\site-packages\xarray_schema\__init__.py:1: UserWarning: pkg_resources is deprecated as an API. See https://setuptools.pypa.io/en/latest/pkg_resources.html. The pkg_resources package is slated for removal as early as 2025-11-30. Refrain from using this package or pin to Setuptools<81.
  from pkg_resources import DistributionNotFound, get_distribution


### Load analysis settings

In [2]:
# load analysis configuration
settings_path = r'C:\sdata_NRR-W\analysis_settings_NRR-W.yaml'

settings = load_analysis_settings(settings_path)
settings

AnalysisConfig(general=GeneralSettings(image_dir='R:/CellDive/NRR-W/NRR-W_Final', analysis_name='NRR-W_Analysis', local_analysis_dir='C:/sdata_NRR-W', remote_analysis_dir='/ix1/kkedziora', log_dir=None), core_detection=CoreDetectionSettings(detection_image='NRR-W_1.0.4_R000_DAPI__FINAL_F.ome.tif', core_info_file_path=None, im_level=6, min_area=2000, max_area=10000, min_iou=0.8, min_st=0.9, min_int=15, frame=4), core_cutting=CoreCuttingSettings(cores_dir_tif=None, cores_dir_output=None, include_channels=None, exclude_channels=None, use_markers=None, ignore_markers=['Antibody1'], margin=0, mask_value=0, transfer_cleanup_enabled=True, core_cleanup_enabled=True), additional_elements=[NormalizeStep(category='image_transformer', type='normalize', input='DAPI', output='DAPI_norm', keep=False, parameters=Params(low=1.0, high=99.8)), NormalizeStep(category='image_transformer', type='normalize', input='ECad', output='ECad_norm', keep=False, parameters=Params(low=1.0, high=99.8)), InstansegStep(c

### Define the logger

In [3]:
log_file = settings.log_dir_path / f"cores_cutting_{datetime.now():%Y-%m-%d_%H-%M-%S}.log"

logger.remove()
l = logger.add(lambda msg: print(msg, end=""))
l = logger.add(log_file, level="DEBUG", enqueue=True)

### Define cores for processing

In [4]:
df_path = settings.core_info_file_path.with_suffix('.pkl')

df = pd.read_pickle(df_path)
df.head()

,core_name,row_start,row_stop,column_start,column_stop,poly_type,polygon_vertices
0,Core_000,192.0,40512.0,90176.0,132736.0,polygon,"[[7424.0, 94528.0], [5824.0, 98432.0], [3392.0..."
1,Core_001,1472.0,39232.0,64.0,26496.0,polygon,"[[9280.0, 8256.0], [6144.0, 11904.0], [2624.0,..."
2,Core_002,1152.0,35584.0,37504.0,70720.0,polygon,"[[22656.0, 37952.0], [17024.0, 37504.0], [1299..."


In [6]:
# # create a subset of cores (optional)
df = df[:1]
df

,core_name,row_start,row_stop,column_start,column_stop,poly_type,polygon_vertices
0,Core_000,192.0,40512.0,90176.0,132736.0,polygon,"[[7424.0, 94528.0], [5824.0, 98432.0], [3392.0..."


## Local files

In [5]:
channel_map = discover_channels(Path(settings.general.image_dir),
                                include_channels=settings.core_cutting.include_channels,
                                exclude_channels=settings.core_cutting.exclude_channels,
                                use_markers=settings.core_cutting.use_markers,
                                ignore_markers=settings.core_cutting.ignore_markers)

2025-11-06 20:49:19.303 | INFO     | multiplex_pipeline.core_cutting.channel_scanner:scan_channels_from_list:80 - Discovered 19 channels:
2025-11-06 20:49:19.305 | INFO     | multiplex_pipeline.core_cutting.channel_scanner:scan_channels_from_list:82 - 001_AP2B <- R:\CellDive\NRR-W\NRR-W_Final\NRR-W_1.0.4_R000_Cy5_AP2B-AF647_FINAL_AFR_F.ome.tif
2025-11-06 20:49:19.305 | INFO     | multiplex_pipeline.core_cutting.channel_scanner:scan_channels_from_list:82 - 001_DAPI <- R:\CellDive\NRR-W\NRR-W_Final\NRR-W_1.0.4_R000_DAPI__FINAL_F.ome.tif
2025-11-06 20:49:19.306 | INFO     | multiplex_pipeline.core_cutting.channel_scanner:scan_channels_from_list:82 - 001_FOXA1 <- R:\CellDive\NRR-W\NRR-W_Final\NRR-W_1.0.4_R000_Cy3_FOXA1-AF555_FINAL_AFR_F.ome.tif
2025-11-06 20:49:19.307 | INFO     | multiplex_pipeline.core_cutting.channel_scanner:scan_channels_from_list:82 - 002_DAPI <- R:\CellDive\NRR-W\NRR-W_Final\NRR-W_2.0.4_R000_DAPI__FINAL_F.ome.tif
2025-11-06 20:49:19.307 | INFO     | multiplex_pipelin

In [ ]:
# # for tests you can request a small set of channels here
# # then send 'short_map' to the controller instead of 'channel_map'
# selected_keys = ["DAPI", "CD3"]
# short_map = {k: channel_map[k] for k in selected_keys if k in channel_map}
# short_map

{'DAPI': 'R:\\CellDive\\BLCA-1B\\BLCA-1B_Final\\BLCA-1B_1.0.4_R000_DAPI__FINAL_F.ome.tif',
 'CD3': 'R:\\CellDive\\BLCA-1B\\BLCA-1B_Final\\BLCA-1B_11.0.4_R000_Cy3_CD3-AF555_FINAL_AFR_F.ome.tif'}

In [6]:
strategy = LocalFileStrategy()

controller = CorePreparationController(
    metadata_df = df, # df defines which cores to process
    image_paths = channel_map, # defines which channels to use
    temp_dir = settings.cores_dir_tif_path,
    output_dir = settings.cores_dir_output_path,
    file_strategy = strategy,
    margin = settings.core_cutting.margin,
    mask_value = settings.core_cutting.mask_value,
    max_pyramid_levels = settings.sdata_storage.max_pyramid_level,
    chunk_size = settings.sdata_storage.chunk_size,
    downscale = settings.sdata_storage.chunk_size,
    )


controller.run()

2025-11-06 20:49:23.709 | INFO     | multiplex_pipeline.core_cutting.controller:run:117 - Starting controller run loop...
2025-11-06 20:49:23.714 | INFO     | multiplex_pipeline.core_cutting.controller:run:127 - Channel AP2B file available at R:\CellDive\NRR-W\NRR-W_Final\NRR-W_4.0.4_R000_Cy5_AP2B-AF647_FINAL_AFR_F.ome.tif.
2025-11-06 20:50:50.253 | DEBUG    | multiplex_pipeline.core_cutting.controller:cut_channel:93 - Cut and saved core Core_000, channel AP2B.
2025-11-06 20:52:21.032 | DEBUG    | multiplex_pipeline.core_cutting.controller:cut_channel:93 - Cut and saved core Core_001, channel AP2B.
2025-11-06 20:54:43.638 | DEBUG    | multiplex_pipeline.core_cutting.controller:cut_channel:93 - Cut and saved core Core_002, channel AP2B.
2025-11-06 20:54:44.459 | DEBUG    | multiplex_pipeline.core_cutting.controller:cut_channel:100 - Closed file handle for channel AP2B.
2025-11-06 20:54:44.824 | INFO     | multiplex_pipeline.core_cutting.controller:run:127 - Channel DAPI file available a